# Data Pre-Processing

In [1]:
import spacy
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
df = pd.read_pickle('data_merged_revised.pkl')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 971264 entries, 0 to 971279
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   text                971264 non-null  object        
 1   date                971264 non-null  datetime64[ns]
 2   legislative_period  971264 non-null  object        
 3   speaker_name        971264 non-null  object        
 4   speaker_gender      971264 non-null  object        
 5   speaker_role        884850 non-null  object        
 6   speaker_party       969563 non-null  object        
 7   comments            971264 non-null  object        
 8   id                  971264 non-null  object        
 9   speaker_id          971264 non-null  int64         
 10  speech_length       971264 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(8)
memory usage: 88.9+ MB


In [ ]:
#!python -m spacy download de_core_news_sm # downloaded successfully

In [17]:
nlp = spacy.load('de_core_news_sm') 
doc = nlp(df.text[0])
doc

  Sehr geehrter Herr Alterspräsident! So muss ich es jetzt sagen, genau; ich muss mich daran gewöhnen. Nicht lange.  Nicht lange; da haben Sie schon recht. – Liebe Kolleginnen und Kollegen! Meine sehr geehrten Damen und Herren! Die Übernahme der Geschäftsordnung ist eine der ersten wichtigen Entscheidungen, die wir heute zu Beginn der Wahlperiode treffen müssen. Die Geschäftsordnung ist Grundlage für unsere gemeinsame Arbeit. Sie hat sich über viele Wahlperioden bewährt und wird uns auch durch die neue Wahlperiode tragen. Es ist gute Tradition, dass wir die Geschäftsordnung am Anfang einer Wahlperiode mit breiter parlamentarischer Mehrheit übernehmen, und das sollten wir auch heute tun. Sie ist nicht in Stein gemeißelt – das wissen diejenigen, die bereits in der letzten Wahlperiode hier im Bundestag tätig waren –, sondern die Geschäftsordnung ist immer ein, ich sage mal, lebendiges Dokument, das sich den Gegebenheiten des Parlamentarismus, aber auch den Gegebenheiten der Arbeit hier im

In [21]:
def keep_token(token):
    include_pos = ['NOUN', 'PROPN']
    exclude_tokens = ['kollege', 'kollegin', 'abgeordneter', 'redner', 'rednerin', 'staatssekretär', 'staatssekretärin', 'minister', 'ministerin', 'bundesminister', 'dame', 'herr']

    if token.is_alpha and token.ent_type_ != 'PER' and token.pos_ in include_pos and not (token.is_stop or token.lemma_.lower() in exclude_tokens):
       return True
    return False

'alterspräsident übernahme geschäftsordnung entscheidung beginn wahlperiode geschäftsordnung grundlage arbeit wahlperiode wahlperiode tradition geschäftsordnung anfang wahlperiode mehrheit stein wahlperiode bundestag geschäftsordnung dokument gegebenheit parlamentarismus gegebenheit arbeit bundestag geschäftsordnung entwicklung wahlperiode änderung reform verhaltensregeln initiative präsident geschäftsordnung wahlperiode generalüberholung impuls fraktion weg vorschlag opposition geschäftsordnung legislaturperiode regelung ausschuß antrag absenken quor beschlussfähigkeit bundestag geschäftsordnung zustimmung antrag weitergeltung geschäftsordnung änderung nächster wort'

In [22]:
speeches_processed = []

for speech in tqdm(nlp.pipe(df.text, n_process=6, batch_size=64)):
    tokens = [token.lemma_.lower() for token in speech if keep_token(token)]
    speeches_processed.append(tokens)

0it [00:00, ?it/s]

In [23]:
len(speeches_processed)

971264

## Save & load processed speeches

In [5]:
import pickle

In [9]:
# save processed speeches to a file
#with open('speeches_processed.pkl', 'wb') as f:
 #  pickle.dump(speeches_processed, f)

# load speeches_processed
with open('speeches_processed.pkl', 'rb') as f:
   loaded_speeches = pickle.load(f)

In [10]:
len(loaded_speeches)

971264

# Topic Modelling

In [ ]:
#!pip install gensim

In [3]:
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaMulticore
from gensim.models import CoherenceModel

In [11]:
dictionary = Dictionary(loaded_speeches)

In [12]:
dictionary.num_docs

971264

In [13]:
dictionary.filter_extremes(no_below=30, no_above=0.2) #TODO?

In [14]:
len(dictionary)

49455

In [15]:
corpus = [dictionary.doc2bow(doc) for doc in loaded_speeches]

In [186]:
num_topics = 150

In [12]:
lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, iterations=50,
                         num_topics=num_topics, workers=8, passes=15, random_state=20)

In [27]:
lda_model.print_topics()

[(0,
  '0.098*"zusatzfrage" + 0.072*"antwort" + 0.032*"beantwortung" + 0.031*"bundesministerium" + 0.031*"geschäftsbereich" + 0.026*"anlage" + 0.024*"danke" + 0.020*"anfrage" + 0.019*"sitzung" + 0.019*"fragestunde"'),
 (1,
  '0.041*"milliarde" + 0.037*"land" + 0.030*"haushalt" + 0.028*"million" + 0.023*"dm" + 0.019*"bund" + 0.010*"investition" + 0.010*"geld" + 0.010*"bereich" + 0.009*"entwicklung"'),
 (2,
  '0.015*"debatte" + 0.015*"cdu" + 0.013*"haus" + 0.013*"regierung" + 0.012*"parlament" + 0.012*"partei" + 0.011*"bundestag" + 0.011*"rede" + 0.010*"spd" + 0.010*"präsident"'),
 (3,
  '0.059*"kind" + 0.058*"frau" + 0.029*"familie" + 0.016*"bildung" + 0.016*"mensch" + 0.014*"eltern" + 0.013*"schule" + 0.013*"gesellschaft" + 0.013*"jugendliche" + 0.010*"mann"'),
 (4,
  '0.063*"fraktion" + 0.048*"ausschuss" + 0.040*"beratung" + 0.038*"drucksache" + 0.029*"antrag" + 0.028*"beschlussempfehlung" + 0.026*"bündnis" + 0.025*"ausschuß" + 0.021*"grüne" + 0.021*"linke"'),
 (5,
  '0.020*"energie" 

# Coherence Model

## CV Coherence Score

In [ ]:
# between 0 and 1
coherence_model_lda = CoherenceModel(model=lda_model, texts=loaded_speeches, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

## UMass Coherence Score

In [ ]:
# between -14 and 14
corpus_lda = lda_model[corpus]

cm = CoherenceModel(model=lda_model, corpus=corpus_lda, dictionary=dictionary, coherence='u_mass')
cm.get_coherence()

# Visualization with pyLDAvis

In [ ]:
#!pip install pyldavis
import pyLDAvis
import pyLDAvis.gensim_models
import pyLDAvis.gensim

In [ ]:
# create topic distance visualization 
pyLDAvis.enable_notebook()
topic_vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
topic_vis

In [ ]:
# save visualization to html file
pyLDAvis.save_html(topic_vis, f"pyLDAvis/lda_{num_topics}_topics.html")

# Save Topic Model

In [17]:
# save LDA model
with open(f'models/lda_model_{num_topics}.pkl', 'wb') as f:
    pickle.dump(lda_model, f)

# save corpus
with open(f'models/corpus.pkl', 'wb') as f:
    pickle.dump(corpus, f)

# Add Topic Distribution to Dataframe

In [5]:
# load LDA model
with open(f'models/lda_model_150.pkl', 'rb') as f:
    loaded_lda_model = pickle.load(f)

In [6]:
# print topics for saving to text file (topics.txt)
for i in range(0, loaded_lda_model.num_topics):
    print(f"Topic {i}: " + loaded_lda_model.print_topic(i))

Topic 0: 0.036*"studie" + 0.031*"risiko" + 0.029*"mensch" + 0.024*"gesundheit" + 0.023*"krankheit" + 0.021*"maßnahme" + 0.018*"gefahr" + 0.016*"pandemie" + 0.015*"untersuchung" + 0.015*"prävention"
Topic 1: 0.121*"wert" + 0.085*"anspruch" + 0.077*"zweck" + 0.037*"zahlung" + 0.033*"grundstück" + 0.027*"fall" + 0.025*"absprache" + 0.023*"enteignung" + 0.021*"inanspruchnahme" + 0.017*"abwicklung"
Topic 2: 0.040*"opfer" + 0.029*"fall" + 0.028*"straftat" + 0.027*"täter" + 0.025*"gewalt" + 0.020*"strafe" + 0.018*"strafrecht" + 0.015*"schutz" + 0.013*"verbrechen" + 0.013*"strafgesetzbuch"
Topic 3: 0.147*"europa" + 0.066*"union" + 0.031*"deutschland" + 0.020*"mitgliedstaat" + 0.020*"eu" + 0.017*"land" + 0.014*"ebene" + 0.014*"parlament" + 0.013*"brüssel" + 0.010*"weg"
Topic 4: 0.125*"modell" + 0.084*"münchen" + 0.066*"wasser" + 0.064*"stoff" + 0.052*"verwendung" + 0.051*"luft" + 0.050*"wirkung" + 0.026*"gefährdung" + 0.019*"leitlinie" + 0.018*"trinkwasser"
Topic 5: 0.086*"sorge" + 0.073*"vater

In [8]:
with open('corpus.pkl', 'rb') as f:
   corpus = pickle.load(f)

In [9]:
# get topic distribution for every speech
topic_distribution = loaded_lda_model.get_document_topics(corpus, minimum_probability=0.2) # threshold
topics_list = [dict(distribution) for distribution in topic_distribution]

In [10]:

# add topic distribution as new column
df['topic_distribution'] = topics_list

In [11]:
df.head()

,text,date,legislative_period,speaker_name,speaker_gender,speaker_role,speaker_party,comments,id,speaker_id,speech_length,topic_distribution
0,Sehr geehrter Herr Alterspräsident! So muss ...,2021-10-26,20,Gabriele Katzmarek,female,None,SPD,[(Beifall bei der SPD sowie bei Abgeordneten d...,SP-2021-0,1299,2593,{68: 0.35882655}
1,Herr Präsident! Liebe Kolleginnen und Kolleg...,2021-10-26,20,Stefan Müller,male,None,CDU/CSU,[(Beifall bei der CDU/CSU sowie bei Abgeordnet...,SP-2021-1,4028,3796,{68: 0.46776655}
2,Sehr geehrter Herr Alterspräsident Wolfgang ...,2021-10-26,20,Britta Haßelmann,female,None,BÜNDNIS 90/DIE GRÜNEN,[(Beifall beim BÜNDNIS 90/DIE GRÜNEN und bei d...,SP-2021-2,451,4301,{68: 0.26328164}
3,Sehr geehrter Herr Präsident! Meine lieben K...,2021-10-26,20,Marco Buschmann,male,None,FDP,"[(Jan Korte [DIE LINKE]: Oder Jugendweihe!), (...",SP-2021-3,3083,4555,{}
4,Herr Alterspräsident! Lassen Sie mich zunäch...,2021-10-26,20,Stephan Brandner,male,None,AfD,"[(Beifall bei der AfD), (Beifall bei Abgeordne...",SP-2021-4,4055,5996,{68: 0.3120498}


In [12]:
# save new dataframe containing topics
df.to_pickle('data_topics_revised.pkl')